In [1]:
import nltk # natural language tool kits
import numpy
import tensorflow as tf
import json
import os

In [2]:
''' 
Preparing a model
'''
def load_data(json_file='intents.json') :

    with open(json_file) as f:
        dataset = json.load(f)
    
    return dataset

In [3]:
def tokenize_words(dataset) :        
    words = []
    intents_x = []
    for intent in dataset:
        for p in intent['patterns']+intent['responses'] : # consider both Q & A
            #tokens = nltk.word_tokenize(p)
            tokens = nltk.regexp_tokenize(p, "[\w']+") # can remove ?
            words.extend(tokens)
            intents_x.append(tokens)

    intent_tags = []
    intents_y = []
    for intent in dataset:
        for p in intent['patterns']+intent['responses'] : # consider both Q & A
            intents_y.append(intent["tag"])
        if intent['tag'] not in intent_tags:
            intent_tags.append(intent['tag'])
            
    intent_tags = sorted(intent_tags)

    print("\nwords=")
    print(len(words))
    print(words)   
    print("\nintents_x=")  
    print(len(intents_x))
    print(intents_x) 
    print("\nintents_y=")   
    print(len(intents_y))
    print(intents_y)
    print("\nintent_tags=")   
    print(intent_tags)

    return (words, intents_x, intents_y, intent_tags)

In [4]:
def stemming(words) :
    '''
    A word stemmer based on the Lancaster (Paice/Husk) stemming algorithm.
    For finding the base or root of words.
    '''
    
    stemmer = nltk.stem.LancasterStemmer()
    words_tmp = []
    for w in words :
        #if w != '?' : words_tmp.append(stemmer.stem(w.lower()))
        words_tmp.append(stemmer.stem(w.lower()))
            
    root_words = sorted(set(words_tmp)) # set() will remove duplications

    print("\nAfter stemming, root words=") 
    print(len(root_words))
    print(root_words)    
    
    return root_words

In [5]:
def get_train_data(root_words, intents_x, intents_y, intent_tags) :

    stemmer = nltk.stem.LancasterStemmer()

    training = []
    output = []

    for i in range(len(intents_x)):
        bag_of_words = []
        words_tmp = []
        for w in intents_x[i] :
            #if w != '?' : words_tmp.append(stemmer.stem(w.lower()))
            words_tmp.append(stemmer.stem(w.lower()))        
            
        for w in root_words:
            found = 1 if w in words_tmp else 0
            bag_of_words.append(found)
            
        training.append(bag_of_words)

        ''' for y '''
        output_row = [0] * len(intent_tags)
        output_row[intent_tags.index(intents_y[i])] = 1
        output.append(output_row)

    train_x = numpy.array(training)
    train_y = numpy.array(output)   
    
    print(train_x)
    print(train_y)
    
    return (train_x, train_y)

In [6]:
def build_model(train_x, train_y) :
    epochs = 500
    batch_size = 16

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(batch_size, input_shape=(len(train_x[0]),))) # input layer
    model.add(tf.keras.layers.Dense(batch_size)) # hidden layer 1
    model.add(tf.keras.layers.Dense(batch_size)) # hidden layer 2
    model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax')) # output layer

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=0)

    ''' Evaluate accuracy of the model '''
    model.summary() # for debugging
    test_loss, test_acc = model.evaluate(train_x, train_y, verbose=0)
    
    return model

In [7]:
def prepare_model(dataset, model_file="model.chatbot") :
    ''' tokenize '''
    words, intents_x, intents_y, intent_tags = tokenize_words(dataset)
    
    ''' stemming '''
    root_words = stemming(words)

    if os.path.isdir(model_file) or os.path.isfile(model_file):
        ''' Load existing model '''
        model = tf.keras.models.load_model(model_file)
        print("Loading model successfully")
    else :
        ''' Create a new model '''                 
        train_x, train_y = get_train_data(root_words, intents_x, intents_y, intent_tags)
        model = build_model(train_x, train_y)
        model.save(model_file)
    
    return (model, root_words, intent_tags)

In [8]:
''' 
Chatting 
'''
def chat(model, root_words, intent_tags, dataset, mess):
    print("Start talking with the bot (type quit to stop)")
    question = mess
    while True:
        
        if question.lower() == "0": break

        print(root_words) 

        ''' tokenize '''
        #tokens = nltk.word_tokenize(question)
        tokens = nltk.regexp_tokenize(question, "[\w']+")
        print(tokens)

        ''' stemming '''
        stemmer = nltk.stem.LancasterStemmer()
        tokens_tmp = []
        for t in tokens:
            tokens_tmp.append(stemmer.stem(t.lower()))
        print(tokens_tmp)
        
        ''' bag of words '''
        bag_of_words = []
        for w in root_words:
            #found = 1 if w in tokens_tmp else 0
            if w in tokens_tmp :
                found = 1
                print(w)
            else :
                found = 0
            bag_of_words.append(found)
        print(bag_of_words)

        ''' model predict to find a tag '''
        probilities = model.predict(numpy.array([bag_of_words]), verbose=1)
        max_index = numpy.argmax(probilities)
        max_probility = probilities[0][max_index]

        print("\nconfidence: {}".format(max_probility))
        print(intent_tags[max_index])
        
        if max_probility < 0.5 :
            return "\nSorry, I don't know what you mean."
            question = input("Youx:")
            continue
        else :
            found_tag = intent_tags[max_index]

        ''' randomly choose a response according to the found tag '''  
        for intent in dataset:
            if intent['tag'] == found_tag: 
                return numpy.random.choice(intent['responses'])


In [9]:
dataset = load_data()

In [10]:
model, root_words, intent_tags = prepare_model(dataset)


words=
1726
['Hi', 'How', 'are', 'you', 'Hola', 'How', 'do', 'you', 'do', 'Hello', 'Whats', 'up', 'Yooo', 'haiiiiii', 'hai', 'Halo', 'Haloooo', 'Hello', 'Is', 'there', 'anything', 'i', 'can', 'help', "It's", 'great', 'to', 'see', 'you', 'What', 'can', 'I', 'help', 'with', 'you', 'today', 'Hi', 'there', 'how', 'can', 'I', 'help', 'Thankyou', 'thank', 'you', 'thanks', 'Tq', 'Thx', 'It', 'worked', 'Done', 'Glad', 'I', 'could', 'help', 'I', 'am', 'happy', 'I', 'could', 'help', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', 'who', 'are', 'you', 'who', 'you', 'who', 'What', 'are', 'you', 'You', 'what', 'What', 'is', 'you', 'What', 'you', 'Tell', 'me', 'about', 'you', 'I', 'am', 'Anvarsy', 'Your', 'virtual', 'customer', 'service', 'What', 'can', 'i', 'help', 'internet', 'package', 'package', 'Access', 'the', 'internet', 'for', '24', '7', 'with', 'the', 'Anvarsy', 'packages', 'Access', 'to', 'gaming', 'streaming', 'and', 'browsing', 'without', 'ex

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                5872      
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 29)                493       
                                                                 
Total params: 6,909
Trainable params: 6,909
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: model.chatbot\assets


INFO:tensorflow:Assets written to: model.chatbot\assets


In [ ]:
import tkinter as tk
import customtkinter

# Function to display chatbot message
def display_chat_message(message, sender):
    chat_area.configure(state='normal')
    chat_area.tag_config("bot", justify="left")
    chat_area.tag_config("you", justify="right")
    chat_area.insert(tk.END, f"{message}\n", sender)
    chat_area.configure(state='disabled')
    chat_area.yview(tk.END)  # Scroll to the bottom of the chat area

    # Display chatbot message in Jupyter Notebook
    print(f"{sender}: {message}")

# Function to receive user input
def get_user_input():
    user_input = user_entry.get()
    user_entry.delete(0, tk.END)
    return user_input

# Function to handle the "Send" button action
def send_message():
    user_message = get_user_input()

    # Display user message
    display_chat_message(user_message, "you")

    # Exit the loop if the user enters "0"
    if user_message == "0":
        window.destroy()
        return

    # Process the user message and get the response from the chatbot
    bot_response = chat(model, root_words, intent_tags, dataset, user_message)

    # Display the chatbot response
    display_chat_message(bot_response, "bot")

# Create the main window
customtkinter.set_appearance_mode("System")
customtkinter.set_default_color_theme("green")
window = customtkinter.CTk()
window.title("Anvarsy AI")

# Set the window size
window_width = 400
window_height = 730
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()
x = int((screen_width - window_width) / 2)
y = int((screen_height - window_height) / 2)
window.geometry(f"{window_width}x{window_height}+{x}+{y}")

# Set the window background
window.configure(bg="#ffffff")

# Create the label for "Anvarsy"
anvarsy_label = tk.Label(window, font=("Quicksand", 16, "bold"), bg="#282424", fg="#FFF")
anvarsy_label.grid(row=0, column=0, padx=10, pady=10, columnspan=2)

# Create the text area to display messages
chat_area = tk.Text(window, width=47, height=39, wrap=tk.WORD, bg="#3D3B3A", fg="#FFF", spacing2=8)  # Change the background color to black and text color to white
chat_area.tag_configure("bot", font=("Calibri", 12))
chat_area.tag_configure("you", font=("Calibri", 12))
chat_area.configure(state='disabled')
chat_area.grid(row=1, column=0, padx=10, pady=1, columnspan=2)

# Create the entry area for user input
user_entry = tk.Entry(window, width=39, font=("Calibri", 12), bg="#3D3B3A", fg="#FFF", bd=1, highlightthickness=0)  # Set the background color of the entry to match the window background
user_entry.grid(row=2, column=0, padx=0, pady=10)

# Create the "Send" button to send a message
send_button = customtkinter.CTkButton(window, text="Send", command=send_message, font=("Arial", 12), width=30, height=25)  # Set the width and height of the button
send_button.grid(row=2, column=1, padx=0, pady=10)

# Display a welcome message from the chatbot
bot_greeting = "Welcome! I'm Anvarsy, your virtual customer support. Just tell me what I can help you with.\n"
display_chat_message(bot_greeting, "bot")

# Run the main window
window.mainloop()


bot: Welcome! I'm Anvarsy, your virtual customer support. Just tell me what I can help you with.

you: hi there
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consist', 'contact', 'contain', 'could', 'credit', 'cs', 'custom', "customer's", 'cut', 'dam', 'dat', 'day', 'decemb', 'deduc', 'defect', 'demograph', 'detect', 'dev', 'dial', 'dick', 'dickhead

1/1 [==============================] - 0s 15ms/step

confidence: 0.9998593330383301
thanks
bot: 
I am happy I could help :)

you: what is internet package
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consist', 'contact', 'contain', 'could', 'credit', 'cs', 'custom', "customer's", 'cut', 'dam', 'dat', 'day', 'decemb', 'deduc', 'defect', 'demograph',

1/1 [==============================] - 0s 15ms/step

confidence: 0.9091778993606567
pulsa_safe
bot: 
Pulsa safe is a new feature that prevents your device from accessing the internet when using non packages.
You can activate this feature through Anvarsy App, settings, Pulsa Safe.

Anything else i can help you with?

you: my quota kept decreasing
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 

1/1 [==============================] - 0s 16ms/step

confidence: 0.5083405375480652
unsatisfied
bot: 
I am sorry if I disappointed you. You can send your feedback at cs@anvarsy.co.id. You also can type in 'Live Agent' to chat with our agent.

you: you are not helping
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consist', 'contact', 'contain', 'coul

1/1 [==============================] - 0s 14ms/step

confidence: 0.4924335777759552
greeting
bot: 
Sorry, I don't know what you mean.
you: why is the internet slow
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consist', 'contact', 'contain', 'could', 'credit', 'cs', 'custom', "customer's", 'cut', 'dam', 'dat', 'day', 'decemb', 'deduc', 'defect', 'de

...

1/1 [==============================] - 0s 0s/step

confidence: 0.9954255223274231
 registe_card_foreigner
bot: 
Foreigners can register prepaid cards by visiting the Gallery ISP with a personal identity card in the form of a passport/KITAS/KITAP.

you: How long does the registrationn process takes
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consis

1/1 [==============================] - 0s 16ms/step

confidence: 0.756259024143219
id_number_register_problem
bot: 
You may need to contact Dukcapil Directorate General customer service via call center 1500537 (on business days and hours)

you: can i register more than one number at a time
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consist', 'con

1/1 [==============================] - 0s 16ms/step

confidence: 0.9671931266784668
select_Number_feature
bot: 
The select number feature can be done easily through the Anvarsy app or st the Anvarsy galleries.

you: can i change my number
Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consist', 'contact', 'contain', 'could', 'credit', 'cs', 'custom',

1/1 [==============================] - 0s 16ms/step

confidence: 0.7883869409561157
number_change_times
bot: 
Changing numbers through the ISP Mobile application can only be done once for both Prepaid and Postpaid customers.

you: how much is the fee for changing the number?

Start talking with the bot (type quit to stop)
["'live", '000', '0811', '1', '100', '11', '1111', '12', '123', '1500537', '16', '1ã', '2', '20', '200', '2023', '24', '3', '4', '4th', '7', 'a', 'abl', 'about', 'access', 'accord', 'act', 'affair', 'aft', 'ag', 'again', "agent'", 'airpl', 'al', 'allow', 'also', 'am', 'amount', 'an', 'and', 'anoth', 'anvarsy', 'anyth', 'ap', 'apply', 'appolog', 'ar', 'as', 'asshol', 'assist', 'at', 'avail', 'back', 'bal', 'be', 'been', 'being', 'bil', 'bitch', 'block', 'both', 'bring', 'brows', 'busy', 'but', 'by', 'cal', 'can', "can't", 'cannot', 'cant', 'card', 'carry', 'cent', 'chang', 'charg', 'chat', 'check', 'choos', 'civil', 'co', 'com', 'connect', 'consist', 'contact', 'contai

1/1 [==============================] - 0s 16ms/step

confidence: 0.999893307685852
SIM_damaged
bot: 
If your card is damaged, you can replace it for free.
Contact our emaul at cs@anvarsy.id, or visit Anvarsy gallery outlets.

